from sentence_transformers.cross_encoder import CrossEncoder
model = CrossEncoder('model_name_or_path')
scores = model.predict([["My first", "sentence pair"],  
                        ["Second text", "pair"]])# Resources
https://colab.research.google.com/drive/1CJqg1LdNU0eMvcWYjBAE4RtDKysPKBae?usp=sharing

In [ ]:
# !pip download sentence_transformers -d ./sentence_transformers/
# # check files
# ! ls ./sentence_transformers/

In [ ]:
# ! ls ./sentence_transformers/

In [ ]:
# ! pip install sentence_transformers datasets
! pip install --no-index --find-links ./sentence_transformers/ sentence-transformers

In [ ]:
import numpy as np 
import pandas as pd
import os
from pathlib import Path
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from datasets import Dataset,DatasetDict

import math
import logging
from datetime import datetime

import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, models, LoggingHandler, losses, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample

In [ ]:
# logger
logging.basicConfig(
    format="%(asctime)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    level=logging.INFO,
    handlers=[LoggingHandler()],
)

In [ ]:
path = Path('/kaggle/input/us-patent-phrase-to-phrase-matching')
df_train = pd.read_csv(path/'train.csv')
df_test = pd.read_csv(path/'test.csv')

In [ ]:
cpc = pd.read_csv('../input/cpc-codes/titles.csv')
cpc.head()
cpc = cpc.rename(columns = {"code" : "context"})
df_train = pd.merge(df_train, cpc[["context","title"]], on ="context", how = "left")
df_test = pd.merge(df_test, cpc[["context","title"]], on ="context", how = "left")

def clean(x):
    t = x.lower()
    t = t.replace("[",'')
    t = t.replace(";",'')
    t = t.replace(",",'')
    t = t.replace("]",'')
    t = t.replace(":",'')
    return t

df_train['title'] = df_train['title'].apply(lambda x: clean(x))
df_test['title'] = df_test['title'].apply(lambda x: clean(x))

In [ ]:
# # Training Valid Test : 70% 15% 15%
# training = Dataset.from_pandas(df_train).train_test_split(0.3, seed=42)
# patent_train = training['train']

# test = training['test'].train_test_split(0.5, seed=42)
# patent_valid = test['train']
# patent_test = test['test']

# print('Length of Train : ',len(patent_train))
# print('Length of Valid : ',len(patent_valid))
# print('Length of Test : ',len(patent_test))

In [ ]:
# Training Valid : 70% 30%     ( for submission )
training = Dataset.from_pandas(df_train).train_test_split(0.3, seed=42)
patent_train = training['train']
patent_valid = training['test']
patent_test = training['test']

print('Length of Train : ',len(patent_train))
print('Length of Valid : ',len(patent_valid))
print('Length of Test : ',len(patent_test))

In [ ]:
######################
option_name = "submit"
######################
# list: submit

# pretrained_model_name = 'klue/roberta-base'    # Spearman's rank correlation coefficient : 0.6510615211928886
# pretrained_model_name = 'sentence-transformers/paraphrase-MiniLM-L6-v2'     # 0.6820976052338351
pretrained_model_name = 'sentence-transformers/all-MiniLM-L12-v2'     # 0.7273263306643135

sts_num_epochs = 4
train_batch_size = 32

# sts_model_save_path = 'output/training_sts-'+pretrained_model_name.replace("/", "-")+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
sts_model_save_path = 'output/training_sts-'+pretrained_model_name.replace("/", "-")+'-'+option_name

In [ ]:
def make_sts_input_example(dataset):
    ''' 
    Transform to InputExample
    ''' 
    
    input_examples = []
#     for i, data in enumerate(dataset):
#         sentence1 = data['sentence1']
#         sentence2 = data['sentence2']
#         score = (data['labels']['label']) / 5.0  # normalize 0 to 5
#         input_examples.append(InputExampleple(texts=[sentence1, sentence2], label=score))
    for i, data in enumerate(dataset):
#         sentence1 = data['context']
#         sentence2 = data['target']
#         anchor = data['anchor']
        sentence1 = data['anchor'] + " " + data['title']
        sentence2 = data['target'] + " " + data['title']
        score = data['score']
        input_examples.append(InputExample(texts=[sentence1, sentence2], label=score))

    return input_examples
sts_train_examples = make_sts_input_example(patent_train)
sts_valid_examples = make_sts_input_example(patent_valid)
sts_test_examples = make_sts_input_example(patent_test)

In [ ]:
# Train Dataloader
train_dataloader = DataLoader(
    sts_train_examples,
    shuffle=True,
    batch_size=train_batch_size,
)

# Evaluator by sts-validation
dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    sts_valid_examples,
    name="dev",
)

# Evaluator by sts-test
test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    sts_test_examples,
    name="test",
)

In [ ]:
if os.path.exists(sts_model_save_path):
    model = SentenceTransformer(sts_model_save_path)
else:
    # Load Embedding Model
    embedding_model = models.Transformer(
        model_name_or_path=pretrained_model_name, 
        max_seq_length=256,
        do_lower_case=True
    )

    # Only use Mean Pooling -> Pooling all token embedding vectors of sentence.
    pooling_model = models.Pooling(
        embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False,
    )

    model = SentenceTransformer(modules=[embedding_model, pooling_model])
    
    # Use CosineSimilarityLoss
    train_loss = losses.CosineSimilarityLoss(model=model)

    # warmup steps
    warmup_steps = math.ceil(len(sts_train_examples) * sts_num_epochs / train_batch_size * 0.1) #10% of train data for warm-up
    logging.info("Warmup-steps: {}".format(warmup_steps))

    # Training
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        evaluator=dev_evaluator,
        epochs=sts_num_epochs,
        evaluation_steps=int(len(train_dataloader)*0.1),
        warmup_steps=warmup_steps,
        output_path=sts_model_save_path
    )
    model.save(sts_model_save_path)

In [ ]:
# evaluation sts-test
score_lst = []
test_evaluator(model, output_path=sts_model_save_path)

In [ ]:
# # Two lists of sentences
# sentences1 = df_train['anchor'][0:100]
# sentences2 = df_train['target'][0:100]

# #Compute embedding for both lists
# embeddings1 = model_pretrained.encode(sentences1, convert_to_tensor=True)
# embeddings2 = model_pretrained.encode(sentences2, convert_to_tensor=True)

# #Compute cosine-similarits
# cosine_scores = util.cos_sim(embeddings1, embeddings2)

# #Output the pairs with their score
# diff = 0
# for i in range(len(sentences1)):
#     print("{} \t\t {} \t\t Score: {:.4f}, real: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i], df_train['score'][i]))
#     diff += abs(cosine_scores[i][i] - df_train['score'][i])
# diff

In [ ]:
# model_pretrained = SentenceTransformer('all-MiniLM-L6-v2')

# # Two lists of sentences
# sentences1 = df_train['anchor'][0:100]
# sentences2 = df_train['target'][0:100]

# #Compute embedding for both lists
# embeddings1 = model.encode(sentences1, convert_to_tensor=True)
# embeddings2 = model.encode(sentences2, convert_to_tensor=True)

# #Compute cosine-similarits
# cosine_scores = util.cos_sim(embeddings1, embeddings2)

# #Output the pairs with their score
# diff = 0
# for i in range(len(sentences1)):
#     print("{} \t\t {} \t\t Score: {:.4f}, real: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i], df_train['score'][i]))
#     diff += abs(cosine_scores[i][i] - df_train['score'][i])
# diff

In [ ]:
def get_scores(model, df):
    # Two lists of sentences
    sentences1 = df['anchor'] + " " + df['title']
    sentences2 = df['target'] + " " + df['title']

    #Compute embedding for both lists
    embeddings1 = model.encode(sentences1, convert_to_tensor=True)
    embeddings2 = model.encode(sentences2, convert_to_tensor=True)

    #Compute cosine-similarits
    cosine_scores = util.cos_sim(embeddings1, embeddings2)
    
    #Output the pairs with their score
#     for i in range(len(sentences1)):
#         print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))
    
    return np.array([cosine_scores[i][i].cpu().data for i in range(len(cosine_scores))])
#     return [cosine_scores[i][i] for i in range(len(cosine_scores))]
        

In [ ]:
rs = get_scores(model, df_test)
df_test['score'] = rs
df_test.head()

In [ ]:
sample_df = pd.read_csv('../input/us-patent-phrase-to-phrase-matching/sample_submission.csv')
sample_df.score = rs
sample_df.to_csv('submission.csv', index=False)